### This notebook is to simply grab all tiles from each WSI and to visualize them to see how they look like. Since each tiles are 10x magnification 512 by 512 tiles, stitching them to a full WSI size will result in a huge image. However, we also don't want to sacrifice resolution, and thankfully we can stitch them by using ometiff extension and we can visualize them by opening it in QuPath.

In [39]:
import pandas as pd
import os
vipshome = r'C:\Users\Kevin\Documents\vips-dev-8.14\bin' #download libvips for windows, this is the bin path
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import time
from tqdm import tqdm

In [40]:
#metadata of each tile
tile_meta_df = pd.read_csv(r"\\fatherserverdw\Kevin\hubmap\tile_meta.csv")
tile_meta_df

,id,source_wsi,dataset,i,j
0,0006ff2aa7cd,2,2,16896,16420
1,000e79e206b7,6,3,10240,29184
2,00168d1b7522,2,2,14848,14884
3,00176a88fdb0,7,3,14848,25088
4,0033bbc76b6b,1,1,10240,43008
...,...,...,...,...,...
7028,ffd37b5c6598,13,3,13824,21504
7029,ffd3d193c71e,3,2,7680,16896
7030,ffd77e2517af,13,3,15872,28160
7031,ffe3cbb81f72,10,3,15456,23000


In [41]:
#group by wsi, let's try visualizing WSI number 1 in this case.
wsi_df = tile_meta_df[tile_meta_df["source_wsi"] == 1]
wsi_df = wsi_df.sort_values(by=['i', 'j'])
wsi_df

,id,source_wsi,dataset,i,j
2200,4e455f0cb054,1,2,1536,41984
2391,5631a47d5b0c,1,1,1536,42496
1337,2fd7649afbc1,1,1,1536,43008
3939,8e90e6189c6b,1,1,2048,38912
3780,88c95fb9fb14,1,1,2048,39424
...,...,...,...,...,...
704,189064c6a137,1,2,17920,10240
6757,f53f64d220e3,1,2,17920,10752
352,0bc3e1a729e4,1,2,18432,9728
4253,99ddcf7fb9a3,1,2,18432,10240


In [42]:
#gather all x and y coordinates in xlist and ylist
max_x = wsi_df['i'].max()
max_y = wsi_df['j'].max()
image_src = r"\\fatherserverdw\Kevin\hubmap\train"
image_paths = []
xlist = []
ylist = []
for index, row in wsi_df.iterrows():
    image_path = os.path.join(image_src, row['id']+ ".tif")
    x_coor = row["i"]
    y_coor = row["j"]
    image_paths.append(image_path)
    xlist.append(x_coor)
    ylist.append(y_coor)

In [43]:
def stitch_tiles_to_ometiff(imlist,xlist,ylist,ometiff_save_dir,ometiff_name):
    """
    :param imlist: list of all image paths
    :param xlist: list of all x coords gathered above
    :param ylist: list of all y coords gathered above
    :param ometiff_save_dir: save directory for final stitched ometiff image
    :param ometiff_name: name for the final stitched ometiff image
    :return: saves stitched WSI ometiff to desired directory
    """
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')
    start = time.time()
    print("counting number of rows and columns to stitch")
    for idx, filename in enumerate(imlist):
        x = int(xlist[idx])
        y = int(ylist[idx])
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    num_row = int((max_x + 512 - min_x)/512)
    num_col = int((max_y + 512 - min_y)/512)
    array_5d = np.zeros((num_col,num_row, 512, 512, 3), dtype=np.uint8)
    print("stitching images")
    for idx, filename in tqdm(enumerate(imlist), desc="Number of images processed", colour = 'red'):
        x = int(xlist[idx])
        y = int(ylist[idx])
        x = int((x-min_x)/512)
        y = int((y-min_y)/512)
        image = Image.open(filename)
        image = np.array(image)
        array_5d[y,x, :, :, :] = image
    stitched_wsi = np.reshape(array_5d.swapaxes(1,2),(512*num_col,512*num_row,3))
    print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))
    end = time.time()
    print("time it took to create reconstructed wsi is: {} minutes".format((round(end-start)/60),3))
    print("--- saving as ometiff ---")
    start = time.time()
    im = pyvips.Image.new_from_array(stitched_wsi)
    image_height = im.height
    image_bands = im.bands
    im = im.copy()
    im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    im.set_type(pyvips.GValue.gstr_type, "image-description",
                f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{image_bands}"
                    SizeT="1"
                    SizeX="{im.width}"
                    SizeY="{image_height}"
                    SizeZ="1"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")

    im.tiffsave(os.path.join(ometiff_save_dir, ometiff_name), compression="jpeg", tile=True, tile_width=512,
                tile_height=512, pyramid=True, subifd=True)
    end = time.time()
    print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))
    print("ometiff saved successfully!")

In [44]:
stitch_tiles_to_ometiff(imlist = image_paths, xlist = xlist, ylist = ylist,ometiff_save_dir = r'\\fatherserverdw\Kevin\hubmap',ometiff_name = 'wsi_1.ome.tiff')# os.path.join ometiff_save_dir, ometiff_name = full save path

counting number of rows and columns to stitch
stitching images


Number of images processed: 507it [02:46,  3.05it/s]


shape of reconstructed wsi is (44544, 17408, 3)
time it took to create reconstructed wsi is: 2.8 minutes
--- saving as ometiff ---
time it took to save ometiff took 0.48333333333333334 minutes
ometiff saved successfully!
